<a href="https://colab.research.google.com/github/poring3995/Midterm_exam/blob/main/Exam__FP_growth_using_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Process 1 :  in this case I upload my csv file to drive for easily managning, so I use drive.mpunt function from my drive to get the datasets

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Process 2 : pull the data from the path of data source that I keep in my drive with delimiter of ";"
import pandas as pd

data = pd.read_csv('/content/gdrive/My Drive/market-basket.csv',sep = ";")
data.head()



<ipython-input-4-13a7a4fa8fc2>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/gdrive/My Drive/market-basket.csv',sep = ";")


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [5]:
# Process 3 : setting the pyspark for using the FPgrowth
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

In [6]:
# Process 4 : setting pyspark environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark   

In [16]:
#process 5 : delete the dulicate data ( that has the same BillNo + Itemname in the table ) and set sort it by the BillNo and I convert the double valus to string in order to merge the data to basket
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas

from pyspark.sql.types import *

df_schema = StructType([StructField("BillNo", StringType(), True)\
                       ,StructField("Quantity", StringType(), True),
                        StructField("Itemname", StringType(), True),
                       StructField("Date", StringType(), True),
                       StructField("Price", StringType(), True),
                       StructField("CustomerID", StringType(), True),
                       StructField("Country", StringType(), True)])
 

sparkdata = spark.createDataFrame(data,schema=df_schema)
basketdata = sparkdata.dropDuplicates(['Itemname', 'BillNo']).sort('BillNo')
basketdata = basketdata.groupBy("BillNo").agg(F.collect_list("Itemname")).sort('BillNo')
     

In [36]:
# process 6 : use FPgrowth to list all the top of data that has most freq value 
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.006, minConfidence=0.006 ) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets


+-------------+-----+
|        items| freq|
+-------------+-----+
|         [12]|12143|
|          [6]|11355|
|      [6, 12]| 8903|
|          [2]|10630|
|       [2, 6]| 7757|
|   [2, 6, 12]| 5972|
|      [2, 12]| 7185|
|          [4]|10540|
|       [4, 2]| 7819|
|    [4, 2, 6]| 6311|
|[4, 2, 6, 12]| 4928|
|   [4, 2, 12]| 5681|
|       [4, 6]| 8163|
|   [4, 6, 12]| 6578|
|      [4, 12]| 7833|
|          [3]| 8543|
|       [3, 2]| 6989|
|    [3, 2, 6]| 5512|
|[3, 2, 6, 12]| 4144|
|   [3, 2, 12]| 4767|
+-------------+-----+
only showing top 20 rows



In [34]:
# process 7 : convert the result to Pandas in order to save into the excel file

# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()

result_pdf.head()

,items,freq
0,[12],12143
1,[6],11355
2,"[6, 12]",8903
3,[2],10630
4,"[2, 6]",7757


In [35]:
# process 8 : save the result into the excel file 

result_pdf.to_excel('midterm_exam.xlsx')

conclusion :

after I try with the diffrent minSupport and minConfidence values I found that the more minSupport and min confidence require is the less data that it show to the result, since if the minnimum require for support or confidence is higher it will have the less data that pass the requirement check inorder to consider this as the frequent item 
